## Asynchronous scraping

In [2]:
# couldn't find this lib so i had to install it
%pip install aiohttp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import csv
import re

In [4]:
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [10]:
async def scrape_and_save_links(text):
  soup = BeautifulSoup(text, 'html.parser')
  file = open('csv_file.csv', 'a', newline='')
  writer = csv.writer(file, delimiter=',')
  for link in soup.findAll('a', attrs={'href':re.compile('^http')}):
    link = link.get('href')
    writer.writerow([link])
  file.close()

In [6]:
async def fetch(session, url):
  try:
    async with session.get(url) as response:
      text = await response.text()
      task = asyncio.create_task(scrape_and_save_links(text))
      await task
  except Exception as e:
    print(str(e)) 

In [7]:
async def scrape(urls):
  tasks = []
  async with aiohttp.ClientSession as session:
    for url in urls:
      tasks.append(fetch(session,url))
    await asyncio.gather(*tasks)

In [11]:
urls = ['https://analytics.usa.gov/', 'https://www.python.org/', 'https://www.linkedin.com/']
asyncio.run(scrape(urls))

TypeError: 'type' object does not support the asynchronous context manager protocol